## Initialize HTMD

In [1]:
from htmd.ui import *
from moleculekit.util import uniformRandomRotation
import nglview as nv
from nglview.datafiles import PDB, XTC
import configparser
from itertools import groupby
import math

2024-05-14 20:15:55,501 - numexpr.utils - INFO - Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-05-14 20:15:55,503 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049
HTMD Documentation at: https://software.acellera.com/htmd/

New HTMD version (2.3.25) is available. You are currently on (2.3.2).There are several methods to update:    - Create a new conda env. using `conda create -n htmd2.3.25 htmd=2.3.25 -c acellera -c conda-forge`    - Create a brand new conda installation and run `conda install htmd -c acellera -c conda-forge`    - Run: `conda update htmd -c acellera -c conda-forge` (NOT RECOMMENDED!)



## Load variables from Input File

In [2]:
cfg = configparser.RawConfigParser()
cfg.read('midas_acemd.cfg')       # Read file
#print cfg.getboolean('Settings','bla') # Manual Way to acess them
par=dict(cfg.items("Settings"))
for p in par:
    par[p]=par[p].split("#",1)[0].strip() # To get rid of inline comments
globals().update(par)  #Make them availible globally
print (par)
try:
    x_offset = float(x_offset)
    y_offset = float(y_offset)
except:
    print('No offset; Centering the cylinder at protein center of mass')
    x_offset = 0.0
    y_offset = 0.0
try:
    inp = open(ligname + '.pdb')
    inp.close()
except:
    print('No ligands associated with protein')
    ligname = 'None'
try:
    x_size_increase = float(x_size_increase)
except:
    x_size_increase = 0.0
try:
    y_size_increase = float(y_size_increase)
except:
    y_size_increase = 0.0
try:
    z_height_increase = float(z_height_increase)
except:
    z_height_increase = 0.0

{'name': 'RECEPTOR', 'ligname': 'LIG', 'lig': 'LIG', 'lipid': 'POPC', 'coname': 'P01', 'pm': '318.273', 'conc': '10', 'csolv': 'P01', 'zoff': '6.000000', 'zheight': '20.0000', 'offset': '1.000000', 'zoffset': '1.000000', 'x_size_increase': '10.00', 'y_size_increase': '10.00', 'disu': '6 - 106, 80 - 160'}
No offset; Centering the cylinder at protein center of mass


## Check if Packmol Runs

In [3]:
pmol = shutil.which("packmol", mode=os.X_OK)
if not pmol:
    raise FileNotFoundError('packmol not found. You should have packmol installed')

## Calculate Cosolvent Box Dimensions

In [4]:
mol1 = Molecule(name + '.pdb')
#Create variable with protein coordinates
c=mol1.get('coords', sel='protein and z > 0')
#Calculate minimum and maximum coordinates
pmin=np.min(c, axis=0)
pmax=np.max(c, axis=0)
xmin=pmin[0] - x_size_increase/2
xmax=pmax[0] + x_size_increase/2
ymin=pmin[1] - y_size_increase/2
ymax=pmax[1] + y_size_increase/2
zmin=pmin[2]
zmax=pmax[2]
xsize=xmax-xmin 
ysize=ymax-ymin
zl2=zmax + float(zoff)
zl4=zl2 + float(zheight)
print(xsize,ysize,zl2,zl4)

49.64299964904785 46.97999954223633 34.91900062561035 54.91900062561035


## Visualize box

In [5]:
mol1.view('not water', gui=False)
b = VMDBox([xmin + float(x_offset), xmax + float(x_offset), ymin + float(y_offset), ymax + float(y_offset), zl2, zl4])

/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)


## Calculate number of water molecules fitting the box

In [6]:
#Create a box of soley water molecules
smol1 = solvate(mol1, minmax=[[xmin, ymin, zl2],[xmax, ymax, zl4]])
#smol1.view()
#Calculate the number of water molecules fitting the box
numwat=len(smol1.get("resid",sel="name OH2"))
print(numwat)

2024-05-14 20:16:09,430 - htmd.builder.solvate - INFO - Using water pdb file at: /home/akimguseynov/.conda/envs/htmd/lib/python3.10/site-packages/htmd/share/solvate/wat.pdb
2024-05-14 20:16:10,465 - htmd.builder.solvate - INFO - Replicating 1 water segments, 1 by 1 by 1
Solvating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.30it/s]
2024-05-14 20:16:10,929 - htmd.builder.solvate - INFO - 1438 water molecules were added to the system.


1438


## Calculate number of cosolvent molecules

In [7]:
numsol=numwat*18.0528*(int(conc)/100)/float(pm)
cnumsol=int(math.ceil(numsol))
print(cnumsol)

9


## Calculate Packmol boxes dimensions

In [8]:
intxs=int(math.ceil(float(xsize)))
intys=int(math.ceil(float(ysize)))
intzs=int(math.ceil(float(zheight)))
print("Boxes dimensions:", intxs, intys, intzs)

Boxes dimensions: 50 47 20


## Creates input for packmol and runs it to create cosolvent box at desired concentration

In [9]:
output1=conc + "_" + coname + "_box"
packs=output1 + ".inp"
print("""
tolerance 2.5
filetype pdb
output %s.pdb
structure %s.pdb
  number %s
  inside box 0. 0. 0. %s. %s. %s.
end structure
""" %(output1,coname,cnumsol,intxs, intys, intzs), file=open(packs, "w"))
os.system(pmol + '<' + packs)


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.010 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Seed for random number generator:      1234567
  Output file: 10_P01_box.pdb
  Reading coordinate file: P01.pdb
  Number of independent structures:            1
  The structures are: 
  Structure            1 :P01.pdb(          19  atoms)
  Maximum number of GENCAN loops for all molecule packing:          200
  Total number of restrictions:            1
  Distance tolerance:    2.5000000000000000     
  Residue numbering set for structure            1 :           0
  Swap c

0

## Moves the cosolvent box to the correct coordinates

In [10]:
cbox=Molecule(output1 + ".pdb")
movedb=cbox.copy()
movedb.moveBy([xmin + float(x_offset), ymin + float(y_offset), zl2])
movedb.view()
b = VMDBox([xmin + float(x_offset), xmax + float(x_offset), ymin + float(y_offset), ymax + float(y_offset), zl2, zl4])
movedb.write(output1 + "_moved.pdb")
cosolvent=(output1 + "_moved.pdb")
print(cosolvent)

/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)


10_P01_box_moved.pdb


## System setup

In [11]:
# System Setup
prot=Molecule(name + '.pdb')  #needs CYS HG->HG1
prot.set('name','HG1',sel='resname CYS and name HG')
memb=Molecule('membrane_c36.pdb')
memb.filter('resname %s' %(lipid))  #only POPC or embed messes up
prot.set('segid', 'PRO', sel='protein')
prot.set('segid', 'W', sel='water')
prot.set('segid', 'CA', sel='resname CA')
prot.remove('resname ACE')
prot.remove('resname NMA')
#Add ligand in the receptor
if ligname != 'None':
    lig = Molecule(ligname + '.pdb')
    lig.set('segid', 'LIG')
    prot.append(lig)
#Add cosolvent box
csol = Molecule(cosolvent)
csol.set('segid','C1')
prot.append(csol)
mol2=embed(prot,memb)
mol2.view()
mol2.write("embedded_system.pdb")

2024-05-14 20:16:26,813 - moleculekit.molecule - INFO - Removed 10611 atoms. 28810 atoms remaining in the molecule.
2024-05-14 20:16:26,996 - moleculekit.molecule - INFO - Removed 0 atoms. 4758 atoms remaining in the molecule.
2024-05-14 20:16:27,043 - moleculekit.molecule - INFO - Removed 0 atoms. 4758 atoms remaining in the molecule.
/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)


## System solvation

In [12]:
smol2 = solvate(mol2, negz=30, posz=10)
smol2.write('solvated_system.pdb')
smol2.view()

2024-05-14 20:16:35,910 - htmd.builder.solvate - INFO - Using water pdb file at: /home/akimguseynov/.conda/envs/htmd/lib/python3.10/site-packages/htmd/share/solvate/wat.pdb
2024-05-14 20:16:37,049 - htmd.builder.solvate - INFO - Replicating 12 water segments, 2 by 2 by 3
Solvating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:04<00:00,  5.40s/it]
2024-05-14 20:17:45,860 - htmd.builder.solvate - INFO - 27897 water molecules were added to the system.
2024-05-14 20:17:46,125 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2024-05-14 20:17:51,394 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no versi

## Print the ligand and cosolvent parameter and topology filenames

In [13]:
if ligname != 'None':
    ligt="top_" + ligname + ".rtf"
    ligp="par_" + ligname + ".prm"
    print(ligt, ligp)
csolt="top_" + coname + ".rtf"
csolp="par_" + coname + ".prm"
print(csolt, csolp)

top_LIG.rtf par_LIG.prm
top_P01.rtf par_P01.prm


## Create the PSF file

In [14]:
topos  = [csolt,'top/top_all36_prot.rtf','top/top_all36_cgenff.rtf','top/top_all36_lipid.rtf','top/top_water_ions.rtf']
params = [csolp,'par/par_all36m_prot.prm','par/par_all36_cgenff.prm','par/par_all36_lipid.prm','par/par_water_ions.prm']
if ligname != 'None':
    topos = [ligt] + topos
    params = [ligp] + params
plm = charmm.build(smol2, topo=topos, param=params,outdir='build',ionize=True,saltconc=0.154,saltanion='CL',saltcation='NA')

2024-05-14 20:18:11,705 - htmd.builder.charmm - INFO - Writing out segments.
2024-05-14 20:18:23,822 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2024-05-14 20:18:24,412 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2024-05-14 20:18:24,914 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2024-05-14 20:18:25,460 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2024-05-14 20:18:25,972 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2024-05-14 20:18:26,026 - htmd.builder.builder - INFO - 2 disulfide bonds were added


Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'R', resid: 6, insertion: '', segid: 'PRO'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'R', resid: 106, insertion: '', segid: 'PRO'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'R', resid: 80, insertion: '', segid: 'PRO'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'R', resid: 160, insertion: '', segid: 'PRO'>



2024-05-14 20:18:26,700 - htmd.builder.charmm - INFO - Starting the build.
2024-05-14 20:18:27,842 - htmd.builder.charmm - WARNING - Failed to set coordinates for 6 atoms.
2024-05-14 20:18:27,843 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 15 atoms.
2024-05-14 20:18:27,844 - htmd.builder.charmm - WARNING - Please check /home/akimguseynov/WORKDIR/build/log.txt for further information.
2024-05-14 20:18:27,845 - htmd.builder.charmm - INFO - Finished building.
2024-05-14 20:18:30,253 - moleculekit.readers - WARNING - Non-integer values were read from the PDB "serial" field. Dropping PDB values and assigning new ones.
2024-05-14 20:18:30,377 - moleculekit.readers - WARNING - Reading PDB file with more than 99999 atoms. Bond information can be wrong.
2024-05-14 20:18:35,815 - htmd.builder.ionize - INFO - Adding 16 anions + 0 cations for neutralizing and 160 ions for the given salt concentration 0.154 M.
2024-05-14 20:20:09,540 - htmd.builder.charmm - INFO - Writing out s

## View the system (commented: load the system from PSF file)

In [15]:
#Added to restart the notebook
#print(lig)
#cfg = configparser.RawConfigParser()
#cfg.read('midas_acemd.cfg')       
#par=dict(cfg.items("Settings"))
#for p in par:
#    par[p]=par[p].split("#",1)[0].strip() # To get rid of inline comments
#globals().update(par)  #Make them availible globally
#print (par)
#print(lig)
#plm=Molecule('build/structure.pdb')
plm.view()

2024-05-14 20:21:24,544 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)


## Measure cell dimensions and origin, measure protein center of mass

In [16]:
boxc=plm.get('coords')
bpmin=np.min(boxc, axis=0)
bpmax=np.max(boxc, axis=0)
bxmin=bpmin[0] 
bxmax=bpmax[0] 
bymin=bpmin[1]
bymax=bpmax[1] 
bzmin=bpmin[2]
bzmax=bpmax[2]
bxsize=bxmax-bxmin 
bysize=bymax-bymin
bzsize=bzmax-bzmin
print('Dimensions:', bxsize,bysize,bzsize)

bcen=np.average(boxc, axis=0)
bcenx=bcen[0]
bceny=bcen[1]
bcenz=bcen[2]
print('Origin:', bcenx,bceny,bcenz)

Dimensions: 92.906 93.597 141.506
Origin: -4.1833773 -2.5440595 -8.219274


## Measure protein center of mass

In [17]:
protc=plm.get('coords', sel='protein')
protm=plm.get('masses', sel='protein')
print('Atomic masses are:', protm)
protcm=np.average(protc, axis=0, weights=protm)
protcenx=protcm[0]
protceny=protcm[1]
protcenz=protcm[2]
print('Protein COM:', protcenx,protceny,protcenz)

Atomic masses are: [12.011  1.008  1.008 ...  1.008  1.008  1.008]
Protein COM: -1.2823386 1.2996099 -6.2013226


## Create constraints for EQ-01

In [18]:
if ligname == 'None':
    plm.set('beta', '1', sel='resname %s or water or ions or protein or (resname %s and name P O11 O12 O13 O14 C11 C12 C13 C14 C15 H11A H11B H12A H12B H13A H13B H13C H14A H14B H14C H15A H15B H15C)' %(csolv,lipid))
else:
    plm.set('beta', '1', sel='resname %s or resname %s or water or ions or protein or (resname %s and name P O11 O12 O13 O14 C11 C12 C13 C14 C15 H11A H11B H12A H12B H13A H13B H13C H14A H14B H14C H15A H15B H15C)' %(ligname,csolv,lipid))
plm.write('mineq-01.pdb')

2024-05-14 20:25:03,399 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.


## Create constraints for EQ-02

In [19]:
plm.set('beta', '0', sel='all')
if ligname == 'None':
    plm.set('beta', '1', sel='protein')
else:
    plm.set('beta', '1', sel='resname %s or protein' %(ligname))
plm.write('mineq-02.pdb')

2024-05-14 20:25:09,073 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.


## Create constraints for EQ-03 and PROD

In [20]:
plm.set('beta', '0', sel='all')
plm.set('beta', '1', sel='protein and name CA and z > -5 and z < 5')
plm.write('prod.pdb')

2024-05-14 20:25:15,416 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.


## Create input file for EQ-01

In [21]:
text = """
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Melting lipid tails. PME, Constant Volume.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb
outputName         mineq-01

set temperature    310

firsttimestep      0

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################
# Input
paraTypeCharmm	    on
parameters          parameters

# NOTE: Do not set the initial velocity temperature if you 
# have also specified a .vel restart file!
temperature         $temperature

# Periodic Boundary Conditions
# NOTE: Do not set the periodic cell basis if you have also 
# specified an .xsc restart file!
if {1} { 
cellBasisVector1     %s    0.   0.
cellBasisVector2     0.    %s   0.
cellBasisVector3     0.    0.   %s
cellOrigin           %s %s %s
}
wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            1.0  ;# 1fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

# Fixed Atoms Constraint (set PDB beta-column to 1)
if {1} {
fixedAtoms          on
fixedAtomsFile      mineq-01.pdb
fixedAtomsCol       B
fixedAtomsForces    on
}

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################

# Minimization
if {1} {
minimize            1000
reinitvels          $temperature
}

#Equilibration
run 500000;# 0.5 ns
""" %(bxsize, bysize,bzsize,bcenx, bceny, bcenz)

parameter_line = 'parameters          parameters\n'
parameter_line += 'parameters          ' + csolp + '\n'
if ligname != 'None':
    parameter_line += 'parameters          ' + ligp + '\n'
#PLEASE FEEL FREE TO MODIFY IF YOU WANT TO ADD MORE PARAMETERS FOR LIGANDS OR NONCANONIC RESIDUES

text = text.replace('parameters          parameters\n', parameter_line)
with open("mineq-01.conf", "w") as out:
    print(text, file=out)

## Get cosolvent ResIDs

In [22]:
csolec=np.unique(plm.get('resid', sel='resname %s and z > 0' %(csolv)))
#wat=smol3.get('coords', sel='water and z > %s and z < %s' %(mzmin,mzmax))

## Calculate cylinder radius

In [23]:
radius=int(math.floor(np.sqrt((xsize*xsize)+(ysize*ysize)))/2)
print(radius)

34


## Calculate cylinder smaller radius

In [24]:
if xsize > ysize:
        smallr=int(math.ceil(xsize)/2)
else:
        smallr=int(math.ceil(ysize)/2)
print('a guidance value for smaller radius (how it would have been for EC/IC simulation):', smallr)
try:
    smallr = float(smaller_radius)
except:
    print('Smaller radius was not provided, using the guidance value reported above')

a guidance value for smaller radius (how it would have been for EC/IC simulation): 25
Smaller radius was not provided, using the guidance value reported above


## Create collective variable file for EQ-02

In [25]:
print("""
Colvarstrajfrequency    50
Colvarsrestartfrequency 50
""", file=open("cosolvent.col", "w"))

for element in csolec:
    #zio=plm.get('resid', sel='resname %s and resid %s z > 0' %(csolv,element))
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    #rint(resid)
    #print(zio)
    print("""
    ### Mol n. %s ###
    colvar {
       name ec_%s
       upperboundary %s
       upperwall %s
       upperwallconstant 200.0
       lowerboundary 0.0
       lowerwall 0.0
       lowerwallconstant 200.0
       distancexy {
          ref {
             dummyAtom ( %s, %s, 0.000 )
          }
          main {
             atomNumbers {%s}
          }
         axis (0.0, 0.0, 1.0)
       }
    }

    colvar {
       name ecz_%s
       upperboundary %s
       upperwall %s
       lowerboundary 0.0
       lowerwall 0.0
       upperwallconstant 200.0
       lowerwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( %s, %s, %s )
          }
          main {
            atomNumbers {%s}
          }
       }
    }
""" %(element,element,radius,radius,x_offset,y_offset,str(zio).strip('[]'),element,zheight,zheight,x_offset,y_offset,zl2,str(zio).strip('[]')), file=open("cosolvent.col", "a"))

## Create input file for EQ-02

In [26]:
text = """
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Protein constrained. PME, Constant Pressure.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb

outputName         mineq-02

set temperature    310

# Continuing a job from the restart files
if {1} {
set inputname      mineq-01
binCoordinates     ./$inputname.restart.coor
binVelocities      ./$inputname.restart.vel  
extendedSystem	   ./$inputname.restart.xsc
} 

firsttimestep      501000 ;# last step of previous run

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################

# Input
paraTypeCharmm	    on
parameters          parameters

wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

# Constant Pressure Control (variable volume)
if {1} {
useGroupPressure      yes ;# needed for 2fs steps
useFlexibleCell       yes  ;# no for water box, yes for membrane
useConstantArea       no  ;# no for water box, yes for membrane

langevinPiston        on
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  200.
langevinPistonDecay   50.
langevinPistonTemp    $temperature
}

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################

# Put here any custom parameters that are specific to 
# this job (e.g., SMD, TclForces, etc...)

constraints on
consexp 2
consref ./structure.pdb
conskfile mineq-02.pdb       
conskcol B
margin 3

tclforces on
set waterCheckFreq              100
set lipidCheckFreq              100
set allatompdb                  ./structure.pdb
tclForcesScript                 keep_water_out.tcl

colvars on
colvarsConfig cosolvent.col

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################

# Minimization
if {1} {
minimize            500
reinitvels          $temperature
}

#Equilibration
run 1000000 ;# 2.0 ns

"""

parameter_line = 'parameters          parameters\n'
parameter_line += 'parameters          ' + csolp + '\n'
if ligname != 'None':
    parameter_line += 'parameters          ' + ligp + '\n'
#PLEASE FEEL FREE TO MODIFY IF YOU WANT TO ADD MORE PARAMETERS FOR LIGANDS OR NONCANONIC RESIDUES

text = text.replace('parameters          parameters\n', parameter_line)
with open("mineq-02.conf", "w") as out:
    print(text, file=out)

## Check if there are gaps in the sequence and adjust the CA accordingly

In [27]:
# Copy cosolvent.col and append a line for the protein
# I need to calculate the gap in the protein sequence
dups = prot.copy()                      # Make a working copy
dups.filter("name CA and protein")      # Keep only C-alphas
rid = dups.get('resid')                 # Get list of residue ids
nrid, count= np.unique(rid,return_counts=True) # Get how many times each residue id appeared
print('Please deal with duplicate ResIDs separately as it may cause errors. The duplicate ResIDs are (if any):')
nrid[count>1]                           # Show duplicates, if any:
deltas = np.diff(rid)
print('Deltas between ResIDs are:', deltas)

#Create the "ca" variable if no gaps in the sequence
ca = str('%s-'%(nrid[0])) #Start the list of ResIDs

#Find gaps in the sequence and adjust the "ca" variable accordingly
new_rid = rid[:np.size(rid)-1] # there is no delta for the last residue

print('Gaps in the sequence are (if any):')
new_rid[deltas!=1]             # Show deltas
# Iterate over all residues excluding last one
gap_counter=0
rn = dups.get('resname')
for i in range(np.size(rid)-1):
    # If there is a break...
    if(deltas[i]>1):
        gap_counter += 1 
        print(gap_counter)
        # Remember that deltas[i]=rid[i+1]-rid[i]
        
        print('Found %s gaps in the sequence' %(gap_counter))
        print(rid[i],rn[i],' followed by ',rid[i+1],rn[i+1])
        print('The gap is %s-%s,'%(rid[i],rid[i+1]))
        print('The sequence except this particular gap is %s-%s,%s-%s'%(rid[0],rid[i],rid[i+1],rid[-1]))
        
        ca += str('%s,%s-'%(nrid[i],nrid[i+1]))

ca += '%s'%(nrid[-1]) #Finish the list of ResIDs
print('Final list of residues is:', ca)

2024-05-14 20:25:37,816 - moleculekit.molecule - INFO - Removed 4667 atoms. 296 atoms remaining in the molecule.


Please deal with duplicate ResIDs separately as it may cause errors. The duplicate ResIDs are (if any):
Deltas between ResIDs are: [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 99  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1

In [28]:
#IF YOU WANT TO SET YOUR OWN LIST OF RESIDS
#ca=('X-Y,A-B')
#print(ca)

## Create collective variable file for EQ-03

In [29]:
shutil.copy2('cosolvent.col', 'cosolvent2.col')
print("""
colvar {
  name Origin
  distance {
     group1 {
      psfSegID PRO
      atomNameResidueRange CA %s
      centerReference on
      refPositions (0,0,0)
     }
     group2 {
     dummyAtom (%s,%s,%s)
     }
  }
}
""" %(ca,protcenx,protceny,protcenz), file=open("cosolvent2.col", "a"))

## Create input file for EQ-03

In [30]:
text = """
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Protein released. PME, Constant Pressure.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb
outputName         eq-03

set temperature    310

# Continuing a job from the restart files
if {1} {
set inputname      mineq-02
binCoordinates     ./$inputname.restart.coor
binVelocities      ./$inputname.restart.vel  
extendedSystem	   ./$inputname.restart.xsc
} 

firsttimestep      1501500 ;# last step of previous run

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################

# Input
paraTypeCharmm	    on
parameters          parameters

wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

# Constant Pressure Control (variable volume)
if {1} {
useGroupPressure      yes ;# needed for 2fs steps
useFlexibleCell       yes  ;# no for water box, yes for membrane
useConstantArea       no  ;# no for water box, yes for membrane

langevinPiston        on
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  200.
langevinPistonDecay   50.
langevinPistonTemp    $temperature
}

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################

# Put here any custom parameters that are specific to 
# this job (e.g., SMD, TclForces, etc...)

constraints on
consexp 2 
consref ./structure.pdb
conskfile prod.pdb
conskcol B
margin 3 

colvars on
colvarsConfig cosolvent2.col

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################

#Equilibration
run 5000000;# 10.0 ns
"""
parameter_line = 'parameters          parameters\n'
parameter_line += 'parameters          ' + csolp + '\n'
if ligname != 'None':
    parameter_line += 'parameters          ' + ligp + '\n'
#PLEASE FEEL FREE TO MODIFY IF YOU WANT TO ADD MORE PARAMETERS FOR LIGANDS OR NONCANONIC RESIDUES

text = text.replace('parameters          parameters\n', parameter_line)
with open("eq-03.conf", "w") as out:
    print(text, file=out)

## Create collective variable file for production

In [31]:
print("""
Colvarstrajfrequency    50
Colvarsrestartfrequency 50
""", file=open("cosolvent3.col", "w"))

for element in csolec:
    #zio=plm.get('resid', sel='resname %s and resid %s z > 0' %(csolv,element))
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    #rint(resid)
    #print(zio)
    print("""
    ### Mol n. %s ###
    colvar {
       name ec_%s
       upperboundary %s
       upperwall %s
       upperwallconstant 200.0
       lowerboundary 0.0
       lowerwall 0.0
       lowerwallconstant 200.0
       distancexy {
          ref {
             dummyAtom ( %s, %s, 0.000 )
          }
          main {
             atomNumbers {%s}
          }
         axis (0.0, 0.0, 1.0)
       }
    }

    colvar {
       name ecz_%s
       upperboundary %s
       upperwall %s
       lowerboundary 0.0
       upperwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( %s, %s, %s )
          }
          main {
            atomNumbers {%s}
          }
       }
    } 
    """ %(element,element,smallr,smallr,x_offset,y_offset,str(zio).strip('[]'),element,zheight,zheight,x_offset,y_offset,zl2,str(zio).strip('[]')), file=open("cosolvent3.col", "a"))
print("""
    colvar {
      name Origin
      distance {
         group1 {
          psfSegID PRO
          atomNameResidueRange CA %s
          centerReference on
          refPositions (0,0,0)
         }
         group2 {
         dummyAtom (%s,%s,%s)
         }
      }
    }
    """%(ca,protcenx,protceny,protcenz), file=open("cosolvent3.col", "a"))

## Create input file for production

In [32]:
text = """
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Protein released. PME, Constant Pressure and Area.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb
outputName         run          

set temperature    310

# Continuing a job from the restart files
if {1} {
set inputname      eq-03
binCoordinates     ./$inputname.restart.coor
binVelocities      ./$inputname.restart.vel  
extendedSystem	   ./$inputname.restart.xsc
} 

firsttimestep      6501500 ;# last step of previous run

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################

# Input
paraTypeCharmm	    on
parameters          parameters

wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

# Constant Pressure Control (variable volume)
if {1} {
useGroupPressure      yes ;# needed for 2fs steps
useFlexibleCell       yes ;# no for water box, yes for membrane
useConstantArea       yes ;# no for water box, yes for membrane

langevinPiston        on
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  200.
langevinPistonDecay   50.
langevinPistonTemp    $temperature
}

restartfreq       10000     ;# 1000steps = every 20ps
dcdfreq           10000
xstFreq            1000
outputEnergies      100
outputPressure      100

#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################

# Put here any custom parameters that are specific to 
# this job (e.g., SMD, TclForces, etc...)

constraints on
consexp 2 
consref ./structure.pdb
conskfile prod.pdb
conskcol B
margin 3  

colvars on
colvarsConfig cosolvent3.col

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################
if {1} { 
  minimize            240
  reinitvels          $temperature
  } 

  run 20000000 ;# 40.0 ns
"""

parameter_line = 'parameters          parameters\n'
parameter_line += 'parameters          ' + csolp + '\n'
if ligname != 'None':
    parameter_line += 'parameters          ' + ligp + '\n'
#PLEASE FEEL FREE TO MODIFY IF YOU WANT TO ADD MORE PARAMETERS FOR LIGANDS OR NONCANONIC RESIDUES

text = text.replace('parameters          parameters\n', parameter_line)
with open("run.conf", "w") as out:
    print(text, file=out)

## Print system specs in a file

In [33]:
print("""
Protein Dimensions: %s %s
Cosolvent Boxes Dimensions: %s %s %s
Number of Cosolvent Molecules: %s
Cell Dimensions: %s %s %s
Cell Origin: %s %s %s
Protein Center of Mass: %s %s %s
Cylinder Radius: %s
Cylinder Z Boundary: %s
Cylinder Smaller Radius: %s
Cylinder X Y: %s %s
""" %(xsize,ysize,intxs,intys,intzs,cnumsol,bxsize,bysize,bzsize,bcenx,bceny,bcenz,protcenx,protceny,protcenz,radius,zl2,smallr,x_offset,y_offset),file=open("system-specs.txt", "w"))